In [ ]:
# uncomment to install the necessary libraries in Colab
#!pip install diffusers[training]

We encourage you to share your model with the community, and in order to do that, you'll need to login to your Hugging Face account (create one [here](https://hf.co/join) if you don't already have one!). You can login from a notebook and enter your token when prompted:

In [ ]:
from dataclasses import dataclass
from datasets import load_dataset, load_from_disk
from torchvision import datasets as dtsts
from torchvision import transforms, utils

from diffusers import DDPMPipeline
import math
import os
from accelerate import Accelerator
from huggingface_hub import HfFolder, Repository, whoami
from tqdm.auto import tqdm
from pathlib import Path
import os
from tqdm.notebook import tqdm


# dataset_name='embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co11'
# dataset_name='embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co25'
dataset_name='o_bc_left_4x_768_360_median_Ultra_Co25'


@dataclass
class TrainingConfig:
    image_size=128
    train_batch_size = 4
    eval_batch_size = 4  # how many images to sample during evaluation
    num_epochs = 100
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 1
    save_model_epochs = 30
    mixed_precision = "no"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = dataset_name  # the model name locally and on the HF Hub

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0


config = TrainingConfig()

In [ ]:
from datasets import load_dataset, load_from_disk
from torchvision import datasets as dtsts

# config.dataset_name = "huggan/smithsonian_butterflies_subset"
# config.dataset_name = "../datasets/original/o_bc_left_4x_768_360_median/"
config.dataset_name = 'data/'+dataset_name
# dataset = load_dataset(config.dataset_name, split="train")
dataset = dtsts.ImageFolder(config.dataset_name)

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose(
    [
        # transforms.Resize((config.image_size, config.image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

In [ ]:
# def transform(examples):
#     images = [preprocess(image.convert("RGB")) for image in examples["image"]]
#     return {"images": images}


# dataset.set_transform(transform)

In [ ]:
import torch

# dataset_path = '../datasets/original/o_bc_left_4x_768_360_median/'
# dataset_path='vqvae2_rosalinity/runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_median/'
dataset_path= 'data/'+dataset_name
# resize_shape = (512, 512)
# resize_shape = (256, 256)

transform = transforms.Compose(
    [
        # transforms.Resize(resize_shape),
        # transforms.CenterCrop(resize_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ]
)

dataset = dtsts.ImageFolder(dataset_path, transform=transform)


train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.train_batch_size, shuffle=True)

In [ ]:
from diffusers import UNet2DModel

model = UNet2DModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=3,  # the number of input channels, 3 for RGB images
    out_channels=3,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)

It is often a good idea to quickly check the sample image shape matches the model output shape:

In [ ]:
sample_image = dataset[0][0].unsqueeze(0)
print("Input shape:", sample_image.shape)

In [ ]:
print("Output shape:", model(sample_image, timestep=0).sample.shape)

## Create a scheduler

In [ ]:
import torch
from PIL import Image
from diffusers import DDPMScheduler

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
noise = torch.randn(sample_image.shape)
timesteps = torch.LongTensor([50])
noisy_image = noise_scheduler.add_noise(sample_image, noise, timesteps)

Image.fromarray(((noisy_image.permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0])

In [ ]:
import torch.nn.functional as F

noise_pred = model(noisy_image, timesteps).sample
loss = F.mse_loss(noise_pred, noise)

## Train the model

In [ ]:
from diffusers.optimization import get_cosine_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_dataloader) * config.num_epochs),
)

In [ ]:


def make_grid(images, rows, cols):
    w, h = images[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, image in enumerate(images):
        grid.paste(image, box=(i % cols * w, i // cols * h))
    return grid


def evaluate(config, epoch, pipeline, model):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.manual_seed(config.seed),
    ).images

    # Make a grid out of the images
    image_grid = make_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join(config.output_dir, "samples")
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")
    
    torch.save(model.state_dict(), f"{test_dir}/{epoch:04d}.pth")

In [ ]:


def get_full_repo_name(model_id: str, organization: str = None, token: str = None):
    if token is None:
        token = HfFolder.get_token()
    if organization is None:
        username = whoami(token)["name"]
        return f"{username}/{model_id}"
    else:
        return f"{organization}/{model_id}"


def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )
    if accelerator.is_main_process:
        # if config.push_to_hub:
        #     repo_name = get_full_repo_name(Path(config.output_dir).name)
        #     repo = Repository(config.output_dir, clone_from=repo_name)
        # elif config.output_dir is not None:
        os.makedirs(config.output_dir, exist_ok=True)
        accelerator.init_trackers("train_example")

    # Prepare everything
    # There is no specific order to remember, you just need to unpack the
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )

    global_step = 0

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch[0]
            # print(clean_images.shape)
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape).to(clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device
            ).long()

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            with accelerator.accumulate(model):
                # Predict the noise residual
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1
        

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline, model)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                if config.push_to_hub:
                    repo.push_to_hub(commit_message=f"Epoch {epoch}", blocking=True)
                else:
                    pipeline.save_pretrained(config.output_dir)

In [ ]:
from accelerate import notebook_launcher

args = (config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

In [ ]:
# model.load_state_dict(torch.load('runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co11/samples/0099.pth'))
model.load_state_dict(torch.load('runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co25/samples/0099.pth'))

model.to('cuda')
pass

In [ ]:
test_dir='runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co25/generated/'

accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )

pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)


In [ ]:
model.load_state_dict(torch.load('runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co25/samples/0099.pth'))

model.to('cuda')
pass


test_dir='runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co25/generated/'

accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )

pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)


step=0
for i in tqdm(range(10)):
    images = pipeline(
        batch_size=10,
        # generator=torch.manual_seed(config.seed),
    ).images

    for image in images:
        # Make a grid out of the images
        image_grid = make_grid([image], rows=1, cols=1)

        image_grid.save(f"{test_dir}/{step}.png")
        step+=1

In [ ]:
model.load_state_dict(torch.load('runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co11/samples/0099.pth'))


model.to('cuda')
pass


test_dir='runs/embs_images_emb_dim_1_n_embed_256_o_bc_left_4x_768_360_768_median_Ultra_Co11/generated/'

accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )

pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)


step=0
for i in tqdm(range(10)):
    images = pipeline(
        batch_size=10,
        # generator=torch.manual_seed(config.seed),
    ).images

    for image in images:
        # Make a grid out of the images
        image_grid = make_grid([image], rows=1, cols=1)

        image_grid.save(f"{test_dir}/{step}.png")
        step+=1